# 构造自己的类型

- 使用 data 关键字定义

In [1]:
data Bool = False | True

类型名和值构造子(Value constructor)的首字母必大写

图形可以是圆 (Circle) 或长方形 (Rectangle), 我们定义一个类型Shape

In [2]:
data Shape = Circle Float Float Float | Rectangle Float Float Float Float

值构造子的本质是个函数，可以返回一个类型的值

我们定义Circle**值构造子**的前两项表示圆心的坐标，尾项表示半径，而Rectangle**值构造子**的前两项为左上角坐标，后两项为右下角坐标

In [3]:
:t Circle
:t Rectangle

Circle :: Float -> Float -> Float -> Shape

Rectangle :: Float -> Float -> Float -> Float -> Shape

写一个函数计算面积

In [4]:
surface :: Shape -> Float
surface (Circle _ _ r) = pi * r ^ 2
surface (Rectangle x1 y1 x2 y2) = (abs $ x1 - x2) * (abs $ y1 - y2)

它的类型声明表示了该函数取一个 Shape 值并返回一个 Float 值。写 Circle -> Float 是不可以的，**因为 Circle 并非类型，真正的类型应该是 Shape**。这与不能写True->False 的道理是一样的。

再就是，我们使用的模式匹配针对的都是值构造子。之前我们匹配过 []、False 或 5，它们都是不包含参数的值构造子。

In [5]:
surface $ Circle 10 20 10
surface $ Rectangle 0 0 100 100

314.15927

10000.0

In [6]:
Circle 10 20

<interactive>:1:1: error:
    • No instance for (Show (Float -> Shape)) arising from a use of ‘print’
        (maybe you haven't applied a function to enough arguments?)
    • In a stmt of an interactive GHCi command: print it

因为 Haskell 还不知道该类型的字符串表示方法。想想，当我们往控制台输出值的时候，Haskell 会先调用 show 函数得到这个值的字符串表示才会输出。因此要让我们的 Shape 类型成为 Show 类型类的成员

In [7]:
data Shape = Circle Float Float Float | Rectangle Float Float Float Float deriving (Show)

In [8]:
Circle 10 20 10

Circle 10.0 20.0 10.0

值构造子就是一个函数，所以可以花样玩：curry 化， map...

取一组不同半径的同心圆，可以这样

In [9]:
map (Circle 10 20) [4,5,6]

[Circle 10.0 20.0 4.0,Circle 10.0 20.0 5.0,Circle 10.0 20.0 6.0]

- 表示点

In [10]:
data Point = Point Float Float deriving (Show)
data Shape = Circle Point Float | Rectangle Point Point deriving (Show)

Point 的定义，它的类型与值构造子用了相同的名字。没啥特殊含义，实际上，在一个类型含有唯一值构造子时这种重名是很常见的

In [11]:
surface :: Shape -> Float
surface (Circle _ r) = pi * r ^ 2
surface (Rectangle (Point x1 y1) (Point x2 y2)) = (abs $ x2 - x1) * (abs $ y2 - y1)

在 Circle 的模式中，我们无视了整个 Point。而在 Rectangle 的模式中，我们用了一个嵌套的模式来取得 Point 中的项。若出于某原因而需要整个 Point，那么直接匹配就是了。

In [12]:
surface (Rectangle (Point 0 0) (Point 100 100))
surface (Circle (Point 0 0) 24)

10000.0

1809.5574

（写起来好像更麻烦了）

- 移动一个图形

In [13]:
nudge :: Shape -> Float -> Float -> Shape
nudge (Circle (Point x y) r) a b = Circle ( Point (x + a, y + b)) r

<interactive>:2:45: error:
    • Couldn't match expected type ‘Point’ with actual type ‘Float -> Point’
    • Probable cause: ‘Point’ is applied to too few arguments
      In the first argument of ‘Circle’, namely ‘(Point (x + a, y + b))’
      In the expression: Circle (Point (x + a, y + b)) r
      In an equation for ‘nudge’: nudge (Circle (Point x y) r) a b = Circle (Point (x + a, y + b)) r
<interactive>:2:51: error:
    • Couldn't match expected type ‘Float’ with actual type ‘(Float, Float)’
    • In the first argument of ‘Point’, namely ‘(x + a, y + b)’
      In the first argument of ‘Circle’, namely ‘(Point (x + a, y + b))’
      In the expression: Circle (Point (x + a, y + b)) r

傻逼了，多加了个逗号

In [14]:
nudge :: Shape -> Float -> Float -> Shape
nudge (Circle (Point x y) r) a b = Circle ( Point (x + a) ( y + b)) r
nudge (Rectangle (Point x1 y1) (Point x2 y2)) a b = Rectangle (Point (x1 +a)(y1+b) ) (Point(x2+a) (y2+b)) 

In [15]:
nudge (Circle (Point 34 34) 10) 5 10

Circle (Point 39.0 44.0) 10.0

- auxilliary function

In [16]:
baseCircle :: Float -> Shape
baseCircle r = Circle (Point 0 0) r

baseRect :: Float -> Float -> Shape
baseRect width height = Rectangle (Point 0 0) (Point width height)

In [17]:
nudge (baseCircle 10) 4 4

Circle (Point 4.0 4.0) 10.0

可以把你的数据类型导出到模块中。只要把你的类型与要导出的函数写到一起就是了。再在后面跟个括号，列出要导出的值构造子，用逗号隔开。如要导出所有的值构造子，那就写个..

In [18]:
module Shapes
( Point(..)
, Shape(..)
, area
, nudge
, baseCircle
, baseRect
) where

import qualified Data.Map as Map

-- data Bool' = False | True

data Point = Point Float Float deriving (Show)
data Shape = Circle Point Float | Rectangle Point Point deriving (Show)

area :: Shape -> Float
area (Circle _ r) = pi * r ^ 2
area (Rectangle (Point x1 y1) (Point x2 y2)) =
	(abs $ x2 - x1) * (abs $ y2 - y1)

nudge :: Shape -> Float -> Float -> Shape
nudge (Circle (Point x y) r) a b = Circle (Point (x+a) (y+b)) r
nudge (Rectangle (Point x1 y1) (Point x2 y2)) a b =
    Rectangle (Point (x1+a) (y1+b)) (Point (x2+a) (y2+b))

nudgedCircle = nudge (Circle (Point 34 34) 10) 5 10

baseCircle :: Float -> Shape
baseCircle r = Circle (Point 0 0) r

baseRect :: Float -> Float -> Shape
baseRect width height = Rectangle (Point 0 0) (Point width height)

Line 20: Move brackets to avoid $
Found:
(abs $ x2 - x1) * (abs $ y2 - y1)
Why not:
(abs $ x2 - x1) * abs (y2 - y1)Line 20: Move brackets to avoid $
Found:
(abs $ x2 - x1) * (abs $ y2 - y1)
Why not:
abs (x2 - x1) * (abs $ y2 - y1)Line 30: Eta reduce
Found:
baseCircle r = Circle (Point 0 0) r
Why not:
baseCircle = Circle (Point 0 0)

一个 Shape (..)，我们就导出了 Shape 的所有值构造子。这一来无论谁导入我们的模块，都可以用 Rectangle 和 Circle 值构造子来构造 Shape 了。这与写 Shape(Rectangle,Circle) 等价

可以选择不导出任何 Shape 的值构造子，这一来使用我们模块的人就只能用辅助函数 baseCircle 和 baseRect 来得到 Shape 了

- 示例 （Map）

没有 Map.Map [(1,2),(3,4)]，因为它没有导出任何一个值构造子。但你可以用，像 Map.fromList 这样的辅助函数得到 map。应该记住，值构造子只是函数而已，如果不导出它们，就拒绝了使用我们模块的人调用它们。但可以使用其他返回该类型的函数，来取得这一类型的值。
不导出数据类型的值构造子隐藏了他们的内部实现，令类型的抽象度更高。同时，我们模块的用户也就无法使用该值构造子进行模式匹配了。

# Record Syntax

In [19]:
data Person = Person String String Int Float String String deriving (Show)

In [20]:
let guy = Person "Buddy" "Finklestein" 43 184.2 "526-2928" "Chocolate"
guy

Person "Buddy" "Finklestein" 43 184.2 "526-2928" "Chocolate"

In [21]:
firstName :: Person -> String
firstName (Person firstname _ _ _ _ _) = firstname

In [22]:
firstName guy

"Buddy"

用record syntax会更加简便

In [23]:
data Person = Person { firstName :: String
                     , lastName :: String
                     , age :: Int
                     , height :: Float
                     , phoneNumber :: String
                     , flavor :: String
                     } deriving (Show)

In [24]:
:t age

age :: Person -> Int

用函数从中直接按项取值

In [25]:
data Car = Car {company :: String, model :: String, year :: Int} deriving (Show)

In [26]:
Car {company="Ford", model="Mustang", year=1967}

Car {company = "Ford", model = "Mustang", year = 1967}

如果一个值构造子中若含有很多个项且不易区分，如一个人或者一辆车啥的，就应该使用 Record Syntax

# Type parameters

Car 的构造子是取三个参数返回一个 Car, 类型构造子可以取类型作参数，产生新的类型

In [27]:
data Maybe a = Nothing | Just a

a就是个类型参数(Type parameter)。也正因为有了它，Maybe 就成为了一个类型构造子(Type constructor，与value constructor对应)

在它的值不是 Nothing 时，它的类型构造子可以搞出 Maybe Int，Maybe String 等等诸多态别。但只一个 Maybe 是不行的，因为它不是类型，而是类型构造子。要成为真正的类型，必须得把它需要的类型参数全部填满

拿 Char 作参数交给 Maybe，就可以得到一个 Maybe Char 的类型。如，Just 'a' 的类型就是 Maybe Char

In [28]:
:t Just "Haha"

Just "Haha" :: Maybe [Char]

一个类型的行为若有点像是容器，那么使用类型参数会是个不错的选择（实现多态）

In [29]:
data Car = Car { company :: String
                 , model :: String
                 , year :: Int
                 } deriving (Show)

In [30]:
tellCar :: Car -> String
tellCar (Car {company = c, model = m, year = y}) = "This " ++ c ++ " " ++ m ++ " was made in " ++ show y

In [31]:
let stang = Car {company="Ford", model="Mustang", year=1967}

In [32]:
tellCar stang

"This Ford Mustang was made in 1967"

有点OOP的感觉

如果Car用类型参数来表示（Car变成了类型构造子），写起来就复杂很多了（因为以前的Car就是类型）

In [33]:
data Car' a b c = Car' { company :: a
                       , model :: b
                       , year :: c
                        } deriving (Show)

In [34]:
tellCar' :: (Show a) => Car' String String a -> String
tellCar' (Car'{company = c, model = m, year = y}) = "This " ++ c ++ " " ++ m ++ " was made in " ++ show y

我们只能强制性地给这个函数安一个 (Show a) => Car String String a 的类型约束。
- 我们不知道a的类型，只能手动去指定类型约束
- 并且，Car只是一个类型构造子，Car String String a 才是一个类型

看得出来，这要繁复得多。而唯一的好处貌似就是，我们可以使用 Show 类型类的 instance 来作 a 的类型

In [35]:
tellCar'  (Car' "Ford" "Mustang" 1967)

"This Ford Mustang was made in 1967"

In [36]:
:t  (Car "Ford" "Mustang" 1967)
:t  (Car' "Ford" "Mustang" 1967)

(Car "Ford" "Mustang" 1967) :: Car

(Car' "Ford" "Mustang" 1967) :: forall c. Num c => Car' [Char] [Char] c

## Data.Map 

Map 中类型参数的使用允许我们能够用一个类型索引另一个类型，只要键的类型在 Ord 类型类就行。如果叫我们自己定义一个 Map 类型，可以在 data 声明中加上一个类型类的约束

In [37]:
-- data (Ord k) => Map' k v...

**Wrong**, **永远不要在 data 声明中添加类型约束**

加了类型约束，每一个与Map相关的函数都要加上类型约束了

toList :: Map k v -> [(k, v)]。要是加上类型约束，就只能是 toList :: (Ord k) =>Map k a -> [(k,v)]，明显没必要嘛

## 小练习： Vector

我们实现个表示三维矢量的类型，再给它加几个处理函数。我么那就给它个类型参数，虽然大多数情况都是数值型，不过这一来它就支持了多种数值类型

In [38]:
data Vector a = Vector a a a deriving (Show)
vplus :: (Num t) => Vector t -> Vector t -> Vector t
(Vector i j k) `vplus` (Vector l m n) = Vector (i+l) (j+m) (k+n)
vectMult :: (Num t) => Vector t -> t -> Vector t
(Vector i j k) `vectMult` m = Vector (i*m) (j*m) (k*m)
scalarMult :: (Num t) => Vector t -> Vector t -> t
(Vector i j k) `scalarMult` (Vector l m n) = i*l + j*m + k*n

注意，声明一个Vector t t t -> Vector t t t->Vector t是错的

Vector 有两种情况：
- 一种是在data定义语句`=`左边的type constructor: data **Vector** a
- 另一种是在data定义语句`=`右边的value constructor **Vector** a a a

声明函数类型时，用的应该是一个类型，也就是Vector a (Vector 是一个类型构造子，接受一个参数a后返回一个类型Vector a)

需要用到值的时候，用的应该是值，也就是用Vector a a a（Vector是一个值构造子，接受3个参数后Vector a a a是一个值）

所以，你不能把一个值放到函数类型定义的语句中去

In [39]:
Vector 3 5 8 `vplus` Vector 9 2 8

Vector 12 7 16

In [40]:
:t (Vector 1 2 3) --Vector 1 2 3是一个值，类型为Vector a
:t (Vector 1.0)

(Vector 1 2 3) :: forall a. Num a => Vector a

(Vector 1.0) :: forall a. Fractional a => a -> a -> Vector a

# Derived instances

类型类(Typeclass) 就是定义了某些行为的接口。例如，Int 类型是 Eq 类型类的一个 instance，Eq 类就定义了判定相等性的行为。Int 值可以判断相等性，所以 Int 就是 Eq 类型类的成员。它的真正威力体现在作为 Eq 接口的函数中，即 == 和 /=。只要一个类型是 Eq 类型类的成员，我们就可以使用 == 函数来处理这一类型。这便是为何 4==4 和 "foo"/="bar" 这样的表达式都需要作类型检查。

类型类更像是接口，我们不是靠它构造数据，而是给既有的数据类型描述行为。什么东西若可以判定相等性，我们就可以让它成为 Eq 类型类的 instance。什么东西若可以比较大小，那就可以让它成为 Ord 类型类的 instance。



In [41]:
data Person = Person { firstName :: String
                     , lastName :: String
                     , age :: Int
                     } deriving (Eq)

Person 类型 derive 为 Eq 的 instance 后, 就可以直接使用 == 或 /= 来判断它们的相等性了。Haskell 会先看下这两个值的值构造子是否一致(这里只是单值构造子)，再用 == 来检查其中的所有数据(必须都是 Eq 的成员)是否一致。

In [42]:
let mikeD = Person {firstName = "Michael", lastName = "Diamond", age = 43}
let adRock = Person {firstName = "Adam", lastName = "Horovitz", age = 41}

In [43]:
mikeD == adRock
mikeD == Person {firstName = "Michael", lastName = "Diamond", age = 43}

False

True

Person 如今已经成为了 Eq 的成员，我们就可以将其应用于所有在类型声明中用到 Eq 类约束的函数了，如 elem

In [44]:
let beastieBoys = [adRock, mikeD]

In [45]:
mikeD `elem` beastieBoys

True

如果我们还没让 Person 类型作为 Show 的成员就尝试输出它，Haskell 就会向我们抱怨，说它不知道该怎么把它表示成一个字符串。不过现在既然已经 derive 成为了 Show 的一个 instance，它就知道了

In [46]:
read "Person {firstName =\"Michael\", lastName =\"Diamond\", age = 43}" :: Person

<interactive>:1:1: error:
    • No instance for (Read Person) arising from a use of ‘read’
    • In the expression: read "Person {firstName =\"Michael\", lastName =\"Diamond\", age = 43}" :: Person
      In an equation for ‘it’: it = read "Person {firstName =\"Michael\", lastName =\"Diamond\", age = 43}" :: Person

In [47]:
data Person' = Person' { firstName :: String
                     , lastName :: String
                     , age :: Int
                     } deriving (Eq, Read, Show)

In [48]:
read "Person {firstName =\"Michael\", lastName =\"Diamond\", age = 43}" :: Person'

Prelude.read: no parse

In [49]:
read "Person' {firstName =\"Michael\", lastName =\"Diamond\", age = 43}" :: Person'

Person' {firstName = "Michael", lastName = "Diamond", age = 43}

In [50]:
--data Bool' = False | True deriving (Ord)

In [51]:
Just 100 `compare` Just 5

<interactive>:1:1: error:
    • No instance for (Ord (Maybe Integer)) arising from a use of ‘compare’
      There are instances for similar types: instance Ord a => Ord (IHaskellPrelude.Maybe a) -- Defined in ‘GHC.Base’
    • In the expression: Just 100 `compare` Just 5
      In an equation for ‘it’: it = Just 100 `compare` Just 5

## 枚举

In [52]:
data Day = Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday

所有的值构造子都是 nullary 的(也就是没有参数)，每个东西都有前置子和后继子，我们可以让它成为 Enum 类型类的成员。同样，每个东西都有可能的最小值和最大值，我们也可以让它成为 Bounded 类型类的成员。在这里，我们就同时将它搞成其它可 derive类型类的 instance

In [53]:
data Day = Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday
           deriving (Eq, Ord, Show, Read, Bounded, Enum)

In [54]:
Wednesday

Wednesday

In [55]:
read "Sunday" :: Day

Sunday

In [56]:
Saturday > Friday

True

In [57]:
minBound:: Day

<interactive>:1:1: error:
    The type signature for ‘minBound’ lacks an accompanying binding
      (The type signature must be given where ‘minBound’ is declared)

- Bug

maxBound :: Int is ambiguous: you could be providing a type signature as a declaration, or it could be an expression. If you write (maxBound :: Int) instead, with parentheses, then it will clearly be an expression.

In [58]:
(minBound :: Day)
(maxBound :: Day)

Line 1: Redundant bracket
Found:
(minBound :: Day)
Why not:
minBound :: DayLine 1: Redundant bracket
Found:
(maxBound :: Day)
Why not:
maxBound :: Day

Monday

Sunday

它也是 Enum 的 instance，可以得到前一天和后一天，并且可以对此使用 List 的区间

In [59]:
succ Monday
pred Friday
[Monday .. Friday]

Tuesday

Thursday

[Monday,Tuesday,Wednesday,Thursday,Friday]

# Type synonyms

[Char] 和 String 等价，可以互换。这就是由类型别名实现的。类型别名实际上什么也没做，只是给类型提供了不同的名字，让我们的代码更容易理解

type 关键字，这个关键字有一定误导性，它并不是用来创造新类（这是 data 关键字做的事情），而是给一个既有类型提供一个别名

In [60]:
phoneBook :: [(String,String)]
phoneBook =
    [("betty","555-2938")
    ,("bonnie","452-2928")
    ,("patsy","493-2928")
    ,("lucille","205-2928")
    ,("wendy","939-8282")
    ,("penny","853-2492")
    ]

type Name = String
type PhoneNum = String
type PhoneBook = [(Name, PhoneNum)]

In [61]:
inPhoneBook :: Name -> PhoneNum -> PhoneBook -> Bool
inPhoneBook name num phonebook = (name, num) `elem` phonebook

In [62]:
inPhoneBook "betty" "555-2938" phoneBook

True

## 带参数的类型别名

类型别名也是可以有参数的

In [63]:
type AssocList k v = [(k,v)]

现在一个从关联 List 中按键索值的函数类型可以定义为 (Eq k) => k -> AssocList k v -> Maybe v. AssocList i。AssocList 是个取两个类型做参数生成一个具体类型的类型构造子，如 Assoc Int String 等等

我们可以用不全调用来得到新的函数，同样也可以使用不全调用得到新的类型构造子。同函数一样，用不全的类型参数调用类型构造子就可以得到一个不全调用的类型构造子

In [64]:
import qualified Data.Map as Map
type IntMap = Map.Map Int
type IntMap' v = Map.Map Int v

IntMap 和 IntMap' 都是一个类型构造子

the IntMap type constructor takes one parameter and that is the type of what the integers will point to.

## Either

In [65]:
--data Either a b = Left a | Right b deriving (Eq, Ord, Read, Show)

它有两个值构造子。如果用了 Left，那它内容的类型就是 a；用了 Right，那它内容的类型就是 b。我们可以用它来将可能是两种类型的值封装起来，从里面取值时就同时提供 Left 和 Right 的模式匹配。

In [66]:
Right 20

Right 20

In [67]:
:t Right "20"
:t Left 20

Right "20" :: forall a. Either a [Char]

Left 20 :: forall a b. Num a => Either a b

我们若想知道函数失败的原因，那还得使用 Either a b，用 a 来表示可能的错误的类型，用 b 来表示一个成功运算的类型。从现在开始，错误一律用 Left 值构造子，而结果一律用 Right

- 小练习

一个例子：有个学校提供了不少壁橱，好给学生们地方放他们的 Gun'N'Rose 海报。每个壁橱都有个密码，哪个学生想用个壁橱，就告诉管理员壁橱的号码，管理员就会告诉他壁橱的密码。但如果这个壁橱已经让别人用了，管理员就不能告诉他密码了，得换一个壁橱。我们就用 Data.Map 的一个 Map 来表示这些壁橱，把一个号码映射到一个表示壁橱占用情况及密码的 Tuple 里

In [164]:
import qualified Data.Map as Map

data LockerState = Taken | Free deriving (Show, Eq)

type Code = String
type LockerMap = Map.Map Int (LockerState, Code)

引入了一个新的类型来表示壁橱的占用情况。并为壁橱密码及按号码找壁橱的 Map 分别设置了一个别名。好，现在我们实现这个按号码找壁橱的函数，就用 Either String Code 类型表示我们的结果，因为 lookup 可能会以两种原因失败。橱子已经让别人用了或者压根就没有这个橱子。如果 lookup 失败，就用字符串表明失败的原因

In [165]:
Map.lookup [ (1, (Taken, "11")]

Parse error (line 1, column 31): parse error on input ‘]’

In [69]:
lockerLookup :: Int -> LockerMap -> Either String Code
lockerLookup lockerNum lockerMap = 
    case Map.lookup lockerNum lockerMap of
        Nothing -> Left $ "Locker number " ++ show lockerNum ++ " doesn't exist!"
        Just (state, code) -> if state /= Taken
                                        then Right code
                                        else Left $ "Locker " ++ show lockerNum ++ " is already taken!"

<interactive>:4:9: error:
    • Couldn't match expected type ‘IHaskellPrelude.Maybe (LockerState, Code)’
                  with actual type ‘Maybe a0’
      NB: ‘Maybe’ is defined at <interactive>:1:1-31
          ‘IHaskellPrelude.Maybe’ is defined in ‘GHC.Base’ in package ‘base-4.10.1.0’
    • In the pattern: Nothing
      In a case alternative: Nothing -> Left $ "Locker number " ++ show lockerNum ++ " doesn't exist!"
      In the expression:
        case Map.lookup lockerNum lockerMap of
          Nothing -> Left $ "Locker number " ++ show lockerNum ++ " doesn't exist!"
          Just (state, code) -> if state /= Taken then Right code else Left $ "Locker " ++ show lockerNum ++ " is already taken!"
<interactive>:5:9: error:
    • Couldn't match expected type ‘IHaskellPrelude.Maybe (LockerState, Code)’
                  with actual type ‘Maybe (LockerState, Code)’
      NB: ‘Maybe’ is defined at <interactive>:1:1-31
          ‘IHaskellPrelude.Maybe’ is defined in ‘GHC.Base’ in package ‘base-4.10.1.0’
    • In the pattern: Just (state, code)
      In a case alternative: Just (state, code) -> if state /= Taken then Right code else Left $ "Locker " ++ show lockerNum ++ " is already taken!"
      In the expression:
        case Map.lookup lockerNum lockerMap of
          Nothing -> Left $ "Locker number " ++ show lockerNum ++ " doesn't exist!"
          Just (state, code) -> if state /= Taken then Right code else Left $ "Locker " ++ show lockerNum ++ " is already taken!"

In [70]:
:t Map.lookup

:t LockerMap

Map.lookup :: forall k a. Ord k => k -> Map k a -> Maybe a

<interactive>:1:1: error: Data constructor not in scope: LockerMap

In [71]:
import qualified Data.Map as Map


data LockerState = Taken | Free deriving (Show, Eq)
type Code = String
type LockerMap = Map.Map Int (LockerState, Code)

lockerLookup :: Int -> LockerMap -> Either String Code
lockerLookup lockerNumber map = case Map.lookup lockerNumber map of
    Nothing -> Left $ "Locker " ++ show lockerNumber ++ " doesn't exist!"
    Just (state, code) -> if state /= Taken
			     then Right code
			     else Left $ "Locker " ++ show lockerNumber
                                         ++ " is already taken!"

<interactive>:6:5: error:
    • Couldn't match expected type ‘IHaskellPrelude.Maybe (LockerState, Code)’
                  with actual type ‘Maybe a0’
      NB: ‘Maybe’ is defined at <interactive>:1:1-31
          ‘IHaskellPrelude.Maybe’ is defined in ‘GHC.Base’ in package ‘base-4.10.1.0’
    • In the pattern: Nothing
      In a case alternative: Nothing -> Left $ "Locker " ++ show lockerNumber ++ " doesn't exist!"
      In the expression:
        case Map.lookup lockerNumber map of
          Nothing -> Left $ "Locker " ++ show lockerNumber ++ " doesn't exist!"
          Just (state, code) -> if state /= Taken then Right code else Left $ "Locker " ++ show lockerNumber ++ " is already taken!"
<interactive>:7:5: error:
    • Couldn't match expected type ‘IHaskellPrelude.Maybe (LockerState, Code)’
                  with actual type ‘Maybe (LockerState, Code)’
      NB: ‘Maybe’ is defined at <interactive>:1:1-31
          ‘IHaskellPrelude.Maybe’ is defined in ‘GHC.Base’ in package ‘base-4.10.1.0’
    • In the pattern: Just (state, code)
      In a case alternative: Just (state, code) -> if state /= Taken then Right code else Left $ "Locker " ++ show lockerNumber ++ " is already taken!"
      In the expression:
        case Map.lookup lockerNumber map of
          Nothing -> Left $ "Locker " ++ show lockerNumber ++ " doesn't exist!"
          Just (state, code) -> if state /= Taken then Right code else Left $ "Locker " ++ show lockerNumber ++ " is already taken!"

In [72]:
lockers :: LockerMap
lockers = Map.fromList
    [(100,(Taken, "ZD39I"))
    ,(101,(Free, "JAH3I"))
    ,(102,(Free, "IQSA9"))
    ,(103,(Free, "QOTSA"))
    ,(104,(Taken, "893JJ"))
    ,(105,(Taken, "99292"))
    ]

locker = lockerLookup 101 lockers
locker' = lockerLookup 100 lockers
locker'' = lockerLookup 109 lockers

<interactive>:10:10: error: Variable not in scope: lockerLookup :: Integer -> LockerMap -> t
<interactive>:11:11: error: Variable not in scope: lockerLookup :: Integer -> LockerMap -> t
<interactive>:12:12: error: Variable not in scope: lockerLookup :: Integer -> LockerMap -> t

( IHaskell 的Maybe类型似乎有点问题，先跳过一下）

# Recursive data structures

我们可以说一个 List 的定义是要码是空的 List 或是一个元素，后面用 : 接了另一串 List。


In [73]:
data List a = Empty | Cons a (List a) deriving (Show, Read, Eq, Ord)

In [74]:
data List' a = Empty | Cons { listHead :: a, listTail :: List' a} deriving (Show, Read, Eq, Ord)

In [75]:
Empty

Empty

In [76]:
5 `Cons` Empty

Cons {listHead = 5, listTail = Empty}

In [77]:
4 `Cons` (5 `Cons` Empty)

Cons {listHead = 4, listTail = Cons {listHead = 5, listTail = Empty}}

Empty 代表 []，而 4 `Cons` (5 `Cons` Empty) 就是 4:(5:[])`

In [78]:
infixr 5 :-:
data List a = Empty | a :-: (List a) deriving (Show, Read, Eq, Ord)

- infix

fixity 声明。当我们定义函数成 operator，我们能同时指定 fixity (但并不是必须的)。fixity 指定了他应该是 left-associative 或是 right-associative，还有他的优先级

In [79]:
3 :-: 4 :-: 5 :-: Empty

3 :-: (4 :-: (5 :-: Empty))

In [80]:
infixr 5  .++
(.++) :: List a -> List a -> List a
Empty .++ ys = ys
(x :-: xs) .++ ys = x :-: (xs .++ ys)

In [81]:
let a = 3 :-: 4 :-: 5 :-: Empty
let b = 6 :-: 7 :-: Empty

In [82]:
a .++ b

3 :-: (4 :-: (5 :-: (6 :-: (7 :-: Empty))))

## BST

定义一棵树的结构：他不是一棵空的树就是带有值并含有两棵子树

In [83]:
data Tree a = EmptyTree | Node a (Tree a) (Tree a) deriving (Show, Read, Eq)

插入函数的型态会是 a -> Tree a -> Tree a。他接受一个元素跟一棵树，并回传一棵包含了新元素的新的树

第一个做了一个单节点的树，而第二个插入一个元素到一棵树中

In [84]:
singleton :: a -> Tree a
singleton x = Node x (EmptyTree) (EmptyTree)

treeInsert :: (Ord a) => a -> Tree a -> Tree a
treeInsert x EmptyTree = singleton x
treeInsert x (Node a left right)
    | x == a  = Node x left right --如果我们要安插的元素跟 root 所含有的元素相等，那就直接回传这棵树
    | x < a = Node a (treeInsert x left) right
    | x > a = Node a left (treeInsert x right)

In [85]:
treeElem :: (Ord a) => a -> Tree a -> Bool
treeElem x EmptyTree = False
treeElem x (Node a left right)
    | x == a = True
    | x < a  = treeElem x left
    | x > a = treeElem x right

In [86]:
let nums = [8,6,4,1,7,3,5]
let numsTree = foldr treeInsert EmptyTree nums

In [87]:
numsTree

Node 5 (Node 3 (Node 1 EmptyTree EmptyTree) (Node 4 EmptyTree EmptyTree)) (Node 7 (Node 6 EmptyTree EmptyTree) (Node 8 EmptyTree EmptyTree))

在 foldr 中，treeInsert 是做 folding 操作的函数，而 EmptyTree 是起始的 accumulator，nums 则是要被走遍的 List

In [88]:
8 `treeElem` numsTree
100 `treeElem` numsTree

True

False

# Typeclasses 的第二堂课

在这个章节中，我们会学到如何构造我们自己的 typeclass，并且如何构造这些 typeclass 的 type instance

typeclass 就像是 interface。一个 typeclass 定义了一些行为(像是比较相不相等，比较大小顺序，能否穷举)而我们会把希望满足这些性质的类型定义成这些 typeclass 的 instance。typeclass 的行为是由定义的函数来描述

The behavior of typeclasses is achieved by defining functions or just type declarations that we then implement. So when we say that a type is an instance of a typeclass, we mean that we can use the functions that the typeclass defines with that type.

当我们把一个类型定义成某个 typeclass 的 instance，就表示我们可以对那个类型使用 typeclass 中定义的函数

Eq 这个 typeclass 是描述可以比较相等的事物。他定义了 == 跟 /= 两个函数。如果我们有一个类型 Car，而且对他们做相等比较是有意义的，那把 Car 作成是 Eq 的一个 instance 是非常合理的。

In [89]:
{-- definition of Eq
class Eq a where            -- 'a' must be lowercase. >1 char allowed
    (==) :: a -> a -> Bool
    (/=) :: a -> a -> Bool  -- function type declarations must be provided
    x == y = not (x /= y)
    x /= y = not (x == y)   -- implementation doesn't
-- Equal if not inequal. inequal if not equal
-- Final type of functions will be: Eq a => a -> a -> Bool
-}

class Eq a where，那代表我们定义了一个新的 typeclass 叫做 Eq。

a 是一个类型变量，他代表 a 是任何我们在定义 instance 时的类型。他不一定要叫做 a。他也不一定非要一个字母不可，只要他是小写就好。然后我们又定义了几个函数。我们并不一定要写出函数的本体，不过必须要写出函数的类型声明

总之我们实作了 Eq 中需要定义的函数本体，只是我们定义他的方式是用交互递归的形式。我们描述两个 Eq 的 instance 要相等，那他们就不能不一样，而他们如果不一样，那他们就是不相等。我们其实不必这样写，但很快你会看到这其实是有用的。

- 示例

In [90]:
data TrafficLight = Red | Yellow | Green

虽然可以透过 derive 让它成为 Eq 或 Show 的 instance，但我们打算手工打造。下面展示了如何让一个类型成为 Eq 的 instance

In [91]:
instance Eq TrafficLight where
    Red == Red = True
    Green == Green = True
    Yellow == Yellow = True
    _ == _ = False

instance 这个关键字。class 是用来定义新的 typeclass，而 instance 是用来说明我们要定义某个 typeclass 的 instance。当我们要定义 Eq，我们会写 class Eq a where，其中 a 代表任何型态的类型。

我们可以从 instance 的写法：instance Eq TrafficLight where 看出来。我们会把 a 换成实际的类型。

我们再来写 Show 的 instance。要满足 Show 的 minimal complete definition，我们必须实作 show 函数，他接受一个值并把他转成字符串。

In [92]:
instance Show TrafficLight where
    show Red = "Red light"
    show Yellow = "Yellow light"
    show Green = "Green light"

In [93]:
Red == Red
Red == Yellow
Red `elem` [Red, Yellow, Green]
[Red, Yellow, Green]

True

False

True

[Red light,Yellow light,Green light]

我们希望印出像 "Red light" 这样的字符串，所以我们就必须手动来写出 instance。

- Typeclass subclass

可以把 typeclass 定义成其他 typeclass 的 subclass

In [94]:
-- class (Eq a) => Num a where

可以在很多地方加上 class constraints。这不过就是在 class Num a where 中的 a 上，加上他必须要是 Eq 的 instance 的限制。这基本上就是在说我们在定义一个类型为 Num 之前，必须先为他定义 Eq 的 instance。在某个类型可以被视作 Number 之前，必须先能被比较相不相等其实是蛮合理的。这就是 subclass 在做的事：帮 class declaration 加上限制。也就是说当我们定义 typeclass 中的函数本体时，我们可以缺省 a 是属于 Eq，因此能使用 ==。

- 类型构造子

In [95]:
{--
instance Eq (Maybe m) where
    Just x == Just y = x == y
    Nothing == Nothing = True
    _ == _ = False
--}

(Maybe m) 这边则取代了 a 在 class Eq a where 的位置。尽管 Maybe 不是一个具体的类型。Maybe m 却是。指定一个类型参数（在这边是小写的 m），我们说我们想要所有像是 Maybe m 的都成为 Eq 的 instance

我们用 == 来比较 Maybe 包含的东西，但我们并没有任何保证说 Maybe 装的东西可以是 Eq。这就是为什么我们需要修改我们的 instance 定义

In [96]:
instance (Eq m) => Eq (Maybe m) where
    Just x == Just y = x == y
    Nothing == Nothing = True
    _ == _ = False

这边我们必须要加上限制。在这个 instance 的声明中，我们希望所有 Maybe m 形式的类型都属于 Eq，但只有当 m 也属于 Eq 的时候。这也是 Haskell 在 derive 的时候做的事。

在大部份情形下，在 typeclass 声明中的 class constraints 都是要让一个 typeclass 成为另一个 typeclass 的 subclass。而在 instance 声明中的 class constraint 则是要表达 对于 类型的一种限制

(==) :: Maybe -> Maybe -> Bool 并非合法。但 (==) :: (Eq m) => Maybe m -> Maybe m -> Bool 则是。这是不论我们要定义什么，通用的类型声明都是 (==) :: (Eq a) => a -> a -> Bool

如果你想看看一个 typeclass 有定义哪些 instance。可以在 ghci 中输入 :info YourTypeClass。所以输入 :info Num 会告诉你这个 typeclass 定义了哪些函数，还有哪些类型属于这个 typeclass。:info 也可以查找类型跟类型构造子的信息。如果你输入 :info Maybe。他会显示 Maybe 所属的所有 typeclass。:info 也能告诉你函数的类型声明

In [97]:
:info Maybe

# yes-no typeclass

In [98]:
class YesNo a where
    yesno :: a -> Bool --a肯定是个具体类型

接下来我们来定义一些 instance。对于数字，我们会假设任何非零的数字都会被当作 true，而 0 则当作 false

In [99]:
instance YesNo Int where
    yesno 0 = False
    yesno _ = True

In [100]:
instance YesNo [] where --[]是type constructor 但[a]才是具体类型
    yesno [] = False
    yesno _ = True

<interactive>:1:16: error:
    • Expecting one more argument to ‘[]’
      Expected a type, but ‘[]’ has kind ‘* -> *’
    • In the first argument of ‘YesNo’, namely ‘[]’
      In the instance declaration for ‘YesNo []’

In [101]:
instance YesNo [a] where --[]是type constructor 但[a]才是具体类型
    yesno [] = False
    yesno _ = True

In [102]:
instance YesNo Bool where
    yesno = id

id 是什么？他不过是标准函数库中的一个函数，他接受一个参数并回传相同的东西

我们也让 Maybe a 成为 YesNo 的 instance


In [103]:
instance YesNo (Maybe a) where
    yesno (Just _) = True
    yesno Nothing  = False

这里还是得写出 (Maybe a) 而不是只有 Maybe，毕竟 Maybe -> Bool 的函数并不存在（因为 Maybe 并不是具体类型），而 Maybe a -> Bool 看起来就合理多了。现在有了这个定义，Maybe something 型式的类型都属于 YesNo 了，不论 something 是什么。

In [104]:
instance YesNo (Tree a) where
    yesno EmptyTree = False
    yesno _ = True

In [105]:
instance YesNo TrafficLight where
    yesno Red = False
    yesno _ = True

In [106]:
yesno []

False

In [107]:
yesno['a']

True

In [108]:
:t yesno

yesno :: forall a. YesNo a => a -> Bool

我们来写一个函数来模仿 if statement 的行为，但他是运作在 YesNo 的类型上。

In [109]:
yesnoIf :: (YesNo y) => y -> a -> a -> a
yesnoIf yesnoVal yesResult noResult =
    if yesno yesnoVal then yesResult else noResult

In [110]:
yesnoIf [] "Yes!" "No!"

"No!"

In [111]:
yesnoIf [1] "Yes!" "No!"

"Yes!"

# Functor typeclass


Functor 这个 typeclass，基本上就代表可以被 map over 的事物。听到这个词你可能会联想到 List，因为 map over list 在 Haskell 中是很常见的操作。你没想错，List 的确是属于 Functor 这个 typeclass

In [112]:
:info Functor

In [113]:
{--
class Functor (f :: * -> *) where
  fmap :: (a -> b) -> f a -> f b
  (<$) :: a -> f b -> f a
  {-# MINIMAL fmap #-}
--}

他定义了一个函数 fmap，而且并没有提供一个缺省的实作。

fmap 的类型蛮有趣的。到目前为止的我们看过的 typeclass 中的类型变量都是具体类型。

但现在碰到的 f 并不是一个具体类型（一个像是 Int, Bool 或 Maybe String的类型），而是**接受一个类型参数的类型构造子**。

**Functor f 中的 f是类型构造子**

**fmap 接受一个函数，这个函数从一个类型映射到另一个类型，还接受一个 functor 装有原始的类型，然后会回传一个 functor 装有映射后的类型**。

**fmap takes a function from one type to another and a functor applied with one type and returns a functor applied with another type.**

In [114]:
{--
instance Functor [] where
    fmap = map
    
--}

不是写成 instance Functor [a] where，因为从 fmap :: (a -> b) -> f a -> f b 可以知道 f 是一个类型构造子，他接受一个类型。而 [a] 则已经是一个具体类型（一个拥有某个类型的 List），其中 [] 是一个类型构造子

In [128]:
instance Functor Maybe where
    fmap f (Just x) = Just (f x)
    fmap f Nothing = Nothing   

instance Functor Maybe where 而不是 instance Functor (Maybe m) where，就像我们在写 YesNo 时的 Maybe 一样。

Functor 要的是一个接受一个类型参数的类型构造子而不是一个具体类型。如果你把 f 代换成 Maybe。fmap 就会像 (a -> b) -> Maybe a -> Maybe b。但如果你把 f 代换成 (Maybe m)，那他就会像 (a -> b) -> Maybe m a -> Maybe m b，这看起来并不合理，因为 Maybe 只接受一个类型参数。

In [135]:
fmap (++ " HEY GUYS IM INSIDE THE JUST") (Just "Something serious.")

<interactive>:1:1: error:
    • No instance for (Show (Maybe String)) arising from a use of ‘print’
      There are instances for similar types: instance Show a => Show (IHaskellPrelude.Maybe a) -- Defined in ‘GHC.Show’
    • In a stmt of an interactive GHCi command: print it

In [123]:
fmap (*2) (Just 400)

<interactive>:1:1: error:
    • No instance for (Show (Maybe Integer)) arising from a use of ‘print’
      There are instances for similar types: instance Show a => Show (IHaskellPrelude.Maybe a) -- Defined in ‘GHC.Show’
    • In a stmt of an interactive GHCi command: print it

IHaskell 的Maybe 问题，有空卡可能看

Tree 的类型构造子也刚好接受单一一个类型参数。如果你把 fmap 看作是一个特别为 Tree 写的函数，他的类型声明会长得像这样 (a -> b) -> Tree a -> Tree b。不过我们在这边会用到递归。map over 一棵空的树会得到一棵空的树。map over 一棵非空的树会得到一棵被函数映射过的树，他的 root 会先被映射，然后左右子树都分别递归地被函数映射

In [118]:
instance Functor Tree where
    fmap f EmptyTree = EmptyTree
    fmap f (Node x left right) = Node (f x) (fmap f left) (fmap f right)

In [119]:
fmap (*2) EmptyTree

EmptyTree

In [120]:
fmap (*4) (foldr treeInsert EmptyTree [5,7,3,2,1,7])

Node 28 (Node 4 EmptyTree (Node 8 EmptyTree (Node 12 EmptyTree (Node 20 EmptyTree EmptyTree)))) EmptyTree

Either a b 是一个Functor吗？

Functor 限制类型构造子只能接受一个类型参数，但 Either 却接受两个，可以 partial apply Either，先喂给他一个参数，并把另一个参数当作 free parameter

In [137]:
{--
instance Functor (Either a) where
    fmap f (Right x) = Right (f x)
    fmap f (Left x) = Left x
    --}

我们把 Either a 定义成一个 instance，而不是 Either。那是因为 Either a 是一个接受单一类型参数的类型构造子，而 Either 则接受两个。

如果 fmap 是针对 Either a，那他的类型声明就会像是 (b -> c) -> Either a b -> Either a c，他又等价于 (b -> c) -> (Either a) b -> (Either a) c

碰到一个 Right 的时候会做 map，但在碰到 Left 的时候却不这样做

In [138]:
-- data Either a b = Left a | Right b

对他们两个都做 map 的动作，那 a 跟 b 必须要是相同的类型。也就是说，如果我们的函数是接受一个字符串然后回传另一个字符串，而且 b 是字符串，a 是数字，这样的情形是不可行的。而且从观察 fmap 的类型也可以知道，当他运作在 Either 上的时候，第一个类型参数必须固定，而第二个则可以改变，而其中第一个参数正好就是 Left 用的

- Data.Map 如何定义成functor

Data.Map 中的 Map 也可以被定义成 functor，像是 Map k v 的情况下，fmap 可以用 v -> v' 这样一个函数来 map over Map k v，并回传 Map k v'。

注意到 ' 在这边并没有特别的意思，他只是用来表示他跟另一个东西有点像，只有一点点差别而已。
你可以自己试试看把 Map k 变成 Functor 的一个 instance

 - functor 应该要遵守一些定律，这样他们的一些性质才能被保证

如果我们给 Tree 定义了错误的 functor instance，对 tree 使用 fmap 可能会导致二元搜索树的性质丧失，也就是在 root 左边的节点不再比 root 小，在 root 右边的节点不再比 root 大

# Kind

类型是一个标签，值会把他带着，这样我们就可以推测出他的性质。但类型也有他们自己的标签，叫做 kind。kind 是类型的类型。

 Types are little labels that values carry so that we can reason about the values. But types have their own little labels, called kinds. A kind is more or less the type of a type. T

In [139]:
:k Int

Int :: *

一个 * 代表这个类型是具体类型。一个具体类型是没有任何类型参数，而值只能属于具体类型。而 * 的读法叫做 star 或是 type

In [140]:
:k Maybe

Maybe :: * -> *

Maybe 的类型构造子接受一个具体类型（像是 Int）然后回传一个具体类型，像是 Maybe Int。这就是 kind 告诉我们的信息。就像 Int -> Int 代表这个函数接受 Int 并回传一个 Int。* -> * 代表这个类型构造子接受一个具体类型并回传一个具体类型。

In [141]:
:k Maybe Int

Maybe Int :: *

对 Maybe 套用了类型参数后会得到一个具体类型（这就是 * -> * 的意思）

In [142]:
:k Either

Either :: * -> * -> *

In [143]:
{--
class Functor f where
    fmap :: (a -> b) -> f a -> f b
    --}


我们看到 f 类型变量是接受一个具体类型且构造出一个具体类型的类型。 知道他构造出具体类型是因为是作为函数参数的类型。 从那里我们可以推测出一个类型要是属于 Functor 必须是 \* -> \* kind

新定义的 typeclass

In [144]:
class Tofu t where
    tofu :: j a -> t a j

由于 j a 被当作 tofu 这个函数的参数的类型，所以 j a 一定是 \* kind。我们假设 a 是 \* kind，那 j 就会是 \* -> \* 的 kind。我们看到 t 由于是函数的回传值，一定是接受两个类型参数的类型。而知道 a 是 \*，j 是\* -> \*，我们可以推测出 t是\* -> (\* -> \*) -> \*。也就是说他接受一个具体类型 a，一个接受单一参数的类型构造子 j，


我们再来定义出一个类型具有 \* -> (\* -> \*) -> \* 的 kind

In [145]:
data Frank a b  = Frank {frankField :: b a} deriving (Show)

Line 1: Use newtype instead of data
Found:
data Frank a b = Frank{frankField :: b a}
                   deriving (Show)
Why not:
newtype Frank a b = Frank{frankField :: b a}
                      deriving (Show)

ADT 中的字段是要来塞值的，所以他们必须是 \* kind。我们假设 a 是 \*，那 b 就是接受一个类型参数的 kind \* -> \*。现在我们知道 a 跟 b 的 kind 了，而他们又是 Frank 的类型参数，所以我们知道 Frank 会有 \* -> (\* -> \*) -> \* 的 kind。第一个 \* 代表 a，而 (\* -> \*) 代表 b

In [149]:
:t Frank{frankField = maybe "haha"}

Frank{frankField = maybe "haha"} :: forall a. Frank (Maybe a -> [Char]) ((->) (a -> [Char]))

In [150]:
:t Frank {frankField = Node 'a' EmptyTree EmptyTree}

Frank {frankField = Node 'a' EmptyTree EmptyTree} :: Frank Char Tree

In [151]:
:t Frank {frankField = "YES"}

Frank {frankField = "YES"} :: Frank Char []

由于结果必须是个值，也就是他必须要是具体类型，因使他必须 fully applied，因此每个 Frank blah blaah 的值都会是 * 的 kind

In [152]:
instance Tofu Frank where
    tofu x  = Frank x

Line 2: Eta reduce
Found:
tofu x = Frank x
Why not:
tofu = Frank

In [154]:
tofu ["HELLO"] :: Frank String []

Frank {frankField = ["HELLO"]}

- 练习2

In [155]:
data Barry t k p = Barry { yabba :: p, dabba :: t k }

我们想要把他定义成 Functor 的 instance。Functor 希望是 \* -> \* 的类型，但 Barry 并不是那种 kind。那 Barry 的 kind 是什么呢？我们可以看到他接受三个类型参数，所以会是 something -> something -> something -> \*。p 是一个具体类型因此是 \*。至于 k，我们假设他是 \*，所以 t 会是 \* -> \*。现在我们把这些代入 something，所以 kind 就变成 (\* -> \*) -> \* -> \* -> \*

In [156]:
:k Barry

Barry :: (* -> *) -> * -> * -> *

现在要把这个类型定义成 Functor，我们必须先 partially apply 头两个类型参数，这样(Barry a b)就会是 \* -> \* 的 kind。这代表 instance 定义会是 instance Functor (Barry a b) where。

如果我们看 fmap 针对 Barry 的类型，也就是把 f 代换成 Barry c d，那就会是 **fmap :: (a -> b) -> (Barry c d) a -> (Barry c d) b**。

第三个 Barry 的类型参数是对于任何类型，所以我们并不牵扯进他

In [157]:
instance Functor (Barry a b) where
    fmap f (Barry {yabba = x, dabba = y}) = Barry {yabba = f x, dabba = y}

Line 2: Redundant bracket
Found:
(Barry{yabba = x, dabba = y})
Why not:
Barry{yabba = x, dabba = y}

In [160]:
:t fmap
:k (Barry _ _)

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

(Barry _ _) :: * -> *

fmap :: (a -> b) -> f a -> f b 

fmap 中，f为 \*->\*, 第二个参数喂给他一个a，然后得到 f b